### 7장: 지시를 따르도록 미세 튜닝하기

In [1]:
from importlib.metadata import version

pkgs = [
    "numpy",
    "matplotlib",
    "tiktoken",
    "torch",
    "tqdm",
    "tensorflow",
] 
for p in pkgs:
    print(f"{p} 버전: {version(p)}")

numpy 버전: 1.26.4
matplotlib 버전: 3.10.7
tiktoken 버전: 0.11.0
torch 버전: 2.6.0
tqdm 버전: 4.67.1
tensorflow 버전: 2.20.0


#### 7.2 지도학습 미세 튜닝을 위한 데이터셋 준비하기

In [2]:
import json
import os
import urllib

def download_and_load_file(file_path, url):
    
    if not os.path.exists(file_path):
        with urllib.request.urlopen(url) as response:
            text_data = response.read().decode("utf-8")
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(text_data)
            
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    
    return data

file_path = "instruction-data.json"
url = (
    "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch"
    "/main/ch07/01_main-chapter-code/instruction-data.json"
)

data = download_and_load_file(file_path, url)
print("샘플 개수:", len(data))

샘플 개수: 1100


- json 파일에서 로드한 data 리스트의 각 항목은 다음 형식의 딕셔너리입니다.

In [3]:
print("샘플 예시:\n", data[50])

샘플 예시:
 {'instruction': 'Identify the correct spelling of the following word.', 'input': 'Ocassion', 'output': "The correct spelling is 'Occasion.'"}


In [4]:
print("다른 샘플:\n", data[999])

다른 샘플:
 {'instruction': "What is an antonym of 'complicated'?", 'input': '', 'output': "An antonym of 'complicated' is 'simple'."}


In [5]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )
    
    input_text = f"\n\n### Input:\n{entry['input']}" if entry['input'] else ""
    
    return instruction_text + input_text

In [6]:
model_input = format_input(data[50])
desired_response = f"\n\n### Response:\n{data[50]['output']}"

print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Identify the correct spelling of the following word.

### Input:
Ocassion

### Response:
The correct spelling is 'Occasion.'


- 입력 필드가 없는 경우

In [8]:
model_input = format_input(data[999])
desired_response = f"\n\n### Response:\n{data[999]['output']}"

print(model_input + desired_response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
What is an antonym of 'complicated'?

### Response:
An antonym of 'complicated' is 'simple'.


- 데이터세트를 훈련, 검증 및 테스트 세트로 나눈다.

In [10]:
train_portion = int(len(data) * 0.85)   # 훈련을 위한 85%
test_portion = int(len(data) * 0.1)     # 테스트를 위한 10
val_portion = len(data) - train_portion - test_portion  # 나머지 5%는 검증용

train_data = data[:train_portion]
test_data = data[train_portion:train_portion + test_portion]
val_data = data[train_portion + test_portion:]

In [11]:
print("훈련 세트 길이:", len(train_data))
print("검증 세트 길이:", len(val_data))
print("테스트 세트 길이:", len(test_data))

훈련 세트 길이: 935
검증 세트 길이: 55
테스트 세트 길이: 110


#### 7.3 훈련 배치 만들기

In [12]:
import torch
from torch.utils.data import Dataset

class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        
        # 텍스트 토큰화
        self.encoded_texts = []
        for entry in data:
            instruction_plus_input = format_input(entry)
            response_text = f"\n\n### Response:\n{entry['output']}"
            full_text = instruction_plus_input + response_text
            self.encoded_texts.append(tokenizer.encode(full_text))
            
    def __getitem__(self, index):
        return self.encoded_texts[index]
    
    def __len__(self):
        return len(self.data)

In [13]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


In [16]:
def custom_collate_draft_1(
    batch,
    pad_token_id=50256,
    device="cpu"
):
    # 배치에서 가장 긴 시퀀스 찾기
    # 그리고, 최대 길이를 +1씩 증가시켜 아래에서 패딩 토큰을 하나 추가합니다.
    batch_max_length = max(len(item)+1 for item in batch) # 최대 길이 시퀀스일 경우 eos 토큰을 넣을 공간 확보
    inputs_lst = []
    
    # 입력 패딩 및 준비
    input_lst = []
    
    for item in batch:
        new_item = item.copy()
        # <|endoftext|> 토큰 추가
        new_item += [pad_token_id]
        
        padded = (
            new_item + [pad_token_id] * 
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1]) # 이전에 추가로 넣은 패팅 토큰을 제외합니다.
        inputs_lst.append(inputs)
        
    inputs_tensor = torch.stack(inputs_lst).to(device)
    return inputs_tensor
    

In [17]:
inputs_1 = [ 0, 1, 2, 3, 4]
inputs_2 = [5, 6]
inputs_3 = [ 7, 8, 9]

batch = (inputs_1, inputs_2, inputs_3)

print(custom_collate_draft_1(batch))

tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])


- 위에서는 LLM에 대한 입력만 반환했습니다. 그러나 LLM 훈련을 위해서는 타킷 값도 필요합니다.
- LLM 사전 훈련과 유사하게, 타깃은 입력을 오른쪽으로 1 위치식 이동한 것이므로 LLM은 다음 토큰을 예측하는 방법을 학습할 수 있습니다.

In [18]:
def custom_collate_draft_2(
    batch,
    pad_token_id=50256,
    device="cpu"
):
    # 배치에서 가장 긴 시퀀스 찾기
    batch_max_length = max(len(item)+1 for item in batch)
    
    # 입력 및 타깃 준비
    inputs_lst, targets_lst = [], []
    
    for item in batch:
        new_item = item.copy()
        # <|endoftext|> 토큰 추가
        new_item += [pad_token_id]
        # 시퀀스를 max_length까지 패딩
        padded = (
            new_item + [pad_token_id] *
            (batch_max_length - len(new_item))
        )
        inputs = torch.tensor(padded[:-1])  # 입력을 위해 마지막 토큰 자르기
        targets = torch.tensor(padded[1:])  # 타깃을 위해 오른쪽으로 +1 이동
        inputs_lst.append(inputs)
        targets_lst.append(targets)
        
    # 입력 리스트를 텐서로 변환하고 타깃 장치로 전송
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)
    
    return inputs_tensor, targets_tensor

In [19]:
inputs, targets = custom_collate_draft_2(batch)
print(inputs)
print(targets)

tensor([[    0,     1,     2,     3,     4],
        [    5,     6, 50256, 50256, 50256],
        [    7,     8,     9, 50256, 50256]])
tensor([[    1,     2,     3,     4, 50256],
        [    6, 50256, 50256, 50256, 50256],
        [    8,     9, 50256, 50256, 50256]])
